In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_data_dir = 'asl_alphabet_train'
test_data_dir = 'asl_alphabet_test'

# Define the number of classes in your dataset
num_classes = 29  # Adjust this to your dataset

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation, including brightness adjustment and horizontal flipping
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[1.0, 1.3]  # Adjust the brightness range as needed
)


In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 87000 images belonging to 29 classes.
Found 28 images belonging to 1 classes.


In [5]:
# Load the MobileNetV2 base model with pre-trained weights (include_top=False)
base_model = MobileNetV2(input_shape=(img_size[0], img_size[1], 3), weights='imagenet', include_top=False)

# Freeze the base model layers to prevent further training
for layer in base_model.layers:
    layer.trainable = False

# Add additional layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)  # Adjust dropout rate as needed
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile the model with your preferred optimizer and loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to review the architecture
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

                                                                                                  
 block_1_expand_relu (ReLU)  (None, 112, 112, 96)         0         ['block_1_expand_BN[0][0]']   
                                                                                                  
 block_1_pad (ZeroPadding2D  (None, 113, 113, 96)         0         ['block_1_expand_relu[0][0]'] 
 )                                                                                                
                                                                                                  
 block_1_depthwise (Depthwi  (None, 56, 56, 96)           864       ['block_1_pad[0][0]']         
 seConv2D)                                                                                        
                                                                                                  
 block_1_depthwise_BN (Batc  (None, 56, 56, 96)           384       ['block_1_depthwise[0][0]']   
 hNormaliz

In [8]:
# Define the number of training epochs
epochs = 10

# Train the model
history = model.fit(train_generator, epochs=epochs, batch_size=batch_size, validation_data=test_generator)

Epoch 1/10
2719/2719 [==============================] - 3259s 1s/step - loss: 0.6943 - accuracy: 0.7886 - val_loss: 543.9078 - val_accuracy: 0.0357
Epoch 2/10
2719/2719 [==============================] - 3329s 1s/step - loss: 0.3819 - accuracy: 0.8767 - val_loss: 649.2408 - val_accuracy: 0.0357
Epoch 3/10
2719/2719 [==============================] - 2981s 1s/step - loss: 0.3263 - accuracy: 0.8931 - val_loss: 751.9849 - val_accuracy: 0.0357
Epoch 4/10
2719/2719 [==============================] - 3372s 1s/step - loss: 0.3048 - accuracy: 0.8999 - val_loss: 800.8012 - val_accuracy: 0.0357
Epoch 5/10
2719/2719 [==============================] - 3605s 1s/step - loss: 0.2794 - accuracy: 0.9071 - val_loss: 808.8665 - val_accuracy: 0.0357
Epoch 6/10
2719/2719 [==============================] - 3563s 1s/step - loss: 0.2636 - accuracy: 0.9132 - val_loss: 900.9352 - val_accuracy: 0.0357
Epoch 7/10
2719/2719 [==============================] - 3594s 1s/step - loss: 0.2559 - accuracy: 0.9165 - val_lo

In [ ]:
model.save('new_model.h5')